In [2]:
import numpy as np
import random

# Определение стратегий
strategies = ["Атаковать", "Защищаться", "Отступить", "Маневрировать", "Переговоры"]
num_strategies = len(strategies)

# Определение числа игроков и начальных альянсов
num_players = 3
alliances = {0: [1], 1: [0], 2: []}  # Игроки 0 и 1 в альянсе
break_threshold = -5  # Порог, при котором альянс разрушается

# Параметры Q-обучения
learning_rate = 0.1
gamma = 0.9  # Коэффициент дисконтирования
epsilon = 0.2  # Эпсилон для эпсилон-жадной стратегии
num_rounds = 10

# Определение типов игроков
player_types = ["агрессивный", "осторожный", "сбалансированный"]

# Инициализация Q-таблиц для каждого игрока
Q_tables = [np.zeros((num_strategies, num_strategies)) for _ in range(num_players)]

# Функция для получения динамической матрицы выплат с учетом внешних факторов
def get_dynamic_payoff_matrix(round_num):
    # Включаем сезонные изменения и случайные факторы
    seasonal_effect = np.sin(round_num / 3.0)  # Сезонный эффект, который изменяется каждую 3-ю итерацию
    random_effect = np.random.normal(0, 1, (num_strategies, num_strategies))
    return np.random.randint(-5, 6, size=(num_strategies, num_strategies)) + seasonal_effect + random_effect

# Функция для пересмотра альянсов
def update_alliances(round_num, payoffs):
    # Пересмотр альянсов: разрушение при уменьшении общих выигрышей
    for i in range(num_players):
        if alliances[i]:
            total_alliance_payoff = sum(payoffs[j] for j in alliances[i])
            if total_alliance_payoff < break_threshold:
                # Разрушаем альянс, если общие выигрыши меньше порога
                for ally in alliances[i]:
                    alliances[ally].remove(i)
                alliances[i] = []

# Начальные стратегии игроков
current_strategies = [random.choice(strategies) for _ in range(num_players)]
print(f"Начальные стратегии игроков: {current_strategies}")

# Проведение нескольких раундов конфликта
for round_num in range(1, num_rounds + 1):
    print(f"\nРаунд {round_num}:")
    
    # Генерация динамической матрицы выплат
    payoff_matrices = [get_dynamic_payoff_matrix(round_num) for _ in range(num_players)]
    
    # Индексы стратегий текущих игроков
    strategy_indices = [strategies.index(strategy) for strategy in current_strategies]
    
    # Получение выплат для каждого игрока
    payoffs = [
        payoff_matrices[player_idx][strategy_indices[player_idx], strategy_indices[(player_idx + 1) % num_players]]
        for player_idx in range(num_players)
    ]
    
    # Печать выплат текущего раунда
    for i in range(num_players):
        print(f"Игрок {i+1} выбрал '{current_strategies[i]}', выплата: {payoffs[i]}")

    # Обновление Q-таблиц
    for i in range(num_players):
        current_state = strategy_indices[i]
        opponent_state = strategy_indices[(i + 1) % num_players]

        # Получение текущего значения Q
        reward = payoffs[i]
        
        # Если игрок в альянсе, добавляем поддержку коалиции
        if alliances[i]:
            alliance_reward = sum(payoffs[j] for j in alliances[i])
            reward += alliance_reward * 0.5

        # Обновление значения Q
        future_q = np.max(Q_tables[i][opponent_state])
        Q_tables[i][current_state, opponent_state] = Q_tables[i][current_state, opponent_state] + \
            learning_rate * (reward + gamma * future_q - Q_tables[i][current_state, opponent_state])

    # Обновление стратегий с использованием эпсилон-жадной стратегии
    for i in range(num_players):
        if random.uniform(0, 1) < epsilon:
            # Выбираем случайную стратегию с вероятностью epsilon
            current_strategies[i] = random.choice(strategies)
        else:
            # Учет типа игрока при выборе стратегии
            q_values = Q_tables[i][strategy_indices[i]]
            if player_types[i] == "агрессивный":
                # Агрессивный игрок выбирает стратегию с наибольшей выплатой
                best_strategy_idx = np.argmax(q_values)
            elif player_types[i] == "осторожный":
                # Осторожный игрок выбирает стратегию с наименьшим риском
                best_strategy_idx = np.argmin(q_values)
            else:
                # Сбалансированный игрок выбирает на основе среднего значения
                best_strategy_idx = np.argmax(q_values) if random.random() > 0.5 else np.argmin(q_values)
            current_strategies[i] = strategies[best_strategy_idx]

    # Пересмотр альянсов после каждого раунда
    update_alliances(round_num, payoffs)

# Итоговые стратегии после всех раундов
print("\nИтоговые стратегии после всех раундов:")
for i in range(num_players):
    print(f"Игрок {i+1}: {current_strategies[i]}")


Начальные стратегии игроков: ['Переговоры', 'Переговоры', 'Маневрировать']

Раунд 1:
Игрок 1 выбрал 'Переговоры', выплата: 4.021979340733754
Игрок 2 выбрал 'Переговоры', выплата: 0.277895606693782
Игрок 3 выбрал 'Маневрировать', выплата: -3.893838890703701

Раунд 2:
Игрок 1 выбрал 'Переговоры', выплата: -0.42505799793360943
Игрок 2 выбрал 'Атаковать', выплата: -3.508639891056381
Игрок 3 выбрал 'Переговоры', выплата: -3.9099060356467428

Раунд 3:
Игрок 1 выбрал 'Переговоры', выплата: -2.856012572970705
Игрок 2 выбрал 'Переговоры', выплата: 7.728539740852771
Игрок 3 выбрал 'Защищаться', выплата: -1.867052631477438

Раунд 4:
Игрок 1 выбрал 'Переговоры', выплата: 5.281989745384807
Игрок 2 выбрал 'Атаковать', выплата: 3.8565305321163574
Игрок 3 выбрал 'Защищаться', выплата: -2.6600696770135404

Раунд 5:
Игрок 1 выбрал 'Переговоры', выплата: -2.481942951783947
Игрок 2 выбрал 'Защищаться', выплата: -0.37186343235524666
Игрок 3 выбрал 'Атаковать', выплата: 6.309121607143597

Раунд 6:
Игрок 1 в